In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from match_image_data import convert_csv_2_dict, load_images_from_folder, get_drone_state



DATAFILE_DRONE = "/home/matthijs/paparazzi/data/AE4317_2019_datasets/cyberzoo_aggressive_flight/20190121-144708.csv"
IMAGES_FOLDER = "/home/matthijs/paparazzi/data/AE4317_2019_datasets/cyberzoo_aggressive_flight/20190121-144646/"
VIDEO_FILE = "/home/matthijs/robotics_q3/mav/cyberzoo.mp4"
VIDEO_FILE_SIM = "/home/matthijs/paparazzi/videos/vlc-record-2023-03-03-11h21m26s-rtp_5000.sdp-.avi"
VIDEO_ALL_OBSTACLES = "/home/matthijs/paparazzi/videos/vlc-record-2023-03-10-10h06m31s-rtp_5000.sdp-.avi"
VIDEO_ORANGE_AVOID = "/home/matthijs/paparazzi/videos/vlc-record-2023-03-10-09h59m06s-rtp_5000.sdp-.avi"
SHOW_CV2_IMG = False
SHOW_VIDEO = True

drone_state_dict = convert_csv_2_dict(DATAFILE_DRONE)
images_cyberzoo_dict = load_images_from_folder(IMAGES_FOLDER)
images_cyberzoo = images_cyberzoo_dict["images"]
images_cyberzoo_stamp = images_cyberzoo_dict["timestamps"]

In [2]:
def show_img(img, gray_scale=False, img_name="img"):
    if SHOW_CV2_IMG:
        cv2.imshow(img_name, img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        if gray_scale:
            cmap = "gray"
        else:
            cmap = None
        plt.imshow(img, cmap=cmap)


In [3]:
def rgb_2_gray(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return gray


In [4]:
# This Optical flow implementation yields to way better results
import time
from collections import deque


def draw_flow(img, flow, step=16):

    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T

    lines = np.vstack([x, y, x-fx, y-fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)

    img_bgr = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.polylines(img_bgr, lines, 0, (0, 255, 0))

    for (x1, y1), (_x2, _y2) in lines:
        cv2.circle(img_bgr, (x1, y1), 1, (0, 255, 0), -1)

    return img_bgr


def draw_hsv(flow):

    h, w = flow.shape[:2]
    fx, fy = flow[:,:,0], flow[:,:,1]

    ang = np.arctan2(fy, fx) + np.pi
    v = np.sqrt(fx*fx+fy*fy)

    hsv = np.zeros((h, w, 3), np.uint8)
    hsv[...,0] = ang*(180/np.pi/2)
    hsv[...,1] = 255
    hsv[...,2] = np.minimum(v*4, 255)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    return bgr

def draw_flow_sides(flow_left, flow_right):
    img_flow_sides = np.zeros((200, 50))
    img_flow_sides[:100, :] = flow_left
    img_flow_sides[100:, :] = flow_right
    return img_flow_sides




if SHOW_VIDEO:

    cap = cv2.VideoCapture(VIDEO_ORANGE_AVOID)

    suc, prev = cap.read()

    prev = prev[160:360,95:145]
    prevgray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
    #prevgray = canny_edge_detection(prevgray)
    flow_left_queue = deque(maxlen=3)
    flow_right_queue = deque(maxlen=3)
    while True:

        suc, img = cap.read()
        if img is None:
            continue 
        img = img[160:360,95:145]
        
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #gray = canny_edge_detection(gray)
        # start time to calculate FPS
        start = time.time()


        flow = cv2.calcOpticalFlowFarneback(prevgray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        flow_left = np.sum(np.linalg.norm(flow[:100,:], axis=2))
        flow_right = np.sum(np.linalg.norm(flow[100:,:], axis=2))
        flow_left_queue.append(flow_left)
        flow_right_queue.append(flow_right)
        sum_flow_left = np.sum(flow_left_queue)
        sum_flow_right = np.sum(flow_right_queue)

        flow_left_norm = sum_flow_left / max(sum_flow_left, sum_flow_right)
        flow_right_norm = sum_flow_right / max(sum_flow_left, sum_flow_right)

        print(flow_left_norm, flow_right_norm)
        prevgray = gray


        # End time
        end = time.time()
        # calculate the FPS for current frame detection
        fps = 1 / (end-start)

        print(f"{fps:.2f} FPS")

        cv2.imshow('flow', draw_flow(gray, flow))
        cv2.imshow('flow sides', draw_flow_sides(flow_left_norm, flow_right_norm))
        #cv2.imshow('flow HSV', draw_hsv(flow))


        key = cv2.waitKey(300)
        if key == ord('q'):
            break


    cap.release()
    cv2.destroyAllWindows()


1.0 0.8871653
491.71 FPS


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/matthijs/.local/lib/python3.10/site-packages/cv2/qt/plugins"


0.9642385 1.0
599.61 FPS
1.0 0.9704773
496.37 FPS
0.99931324 1.0
446.35 FPS
1.0 0.93099725
515.71 FPS
1.0 0.99968046
650.58 FPS
1.0 0.7876371
552.97 FPS
1.0 0.746801
333.36 FPS
1.0 0.7528928
370.16 FPS
1.0 0.95773405
366.99 FPS
1.0 0.98603725
492.58 FPS
1.0 0.92783356
581.17 FPS
1.0 0.8881881
673.89 FPS
1.0 0.89251506
385.22 FPS
1.0 0.7957893
530.39 FPS
1.0 0.89315206
511.63 FPS
1.0 0.73379385
535.67 FPS
1.0 0.83037806
669.91 FPS
1.0 0.8336758
609.73 FPS
0.9645514 1.0
692.93 FPS
0.8860279 1.0
585.80 FPS
0.9214657 1.0
589.50 FPS
1.0 0.64922017
539.32 FPS
1.0 0.60444796
585.39 FPS
1.0 0.5589485
682.78 FPS
1.0 0.5798873
669.27 FPS
1.0 0.49101335
506.13 FPS
1.0 0.36406595
580.77 FPS
1.0 0.12760006
602.37 FPS
1.0 0.11800147
598.76 FPS
1.0 0.18753251
552.83 FPS
1.0 0.35387823
660.10 FPS
1.0 0.470296
557.16 FPS
1.0 0.5555101
395.58 FPS
1.0 0.61674184
402.60 FPS
1.0 0.64667445
391.66 FPS
1.0 0.5361573
334.93 FPS
1.0 0.303747
328.22 FPS
1.0 0.22367215
326.38 FPS
1.0 0.33282843
329.38 FPS
1.0 0.